<center><h1>InterFaceGAN reimplementation</h1></center>

This notebook contains a reimplementation of InterFaceGAN. It uses datasets generated by the notebook `ImageGenerator.ipynb` and labelled using a discriminator network trained on the CelebA dataset for 40 attributes (notebook `ImageDiscriminator.ipynb`).

Imports, and mount Google Drive

Install StyleGAN2-ada-pytorch dependencies

In [1]:
# This pip install avoids warnings.
#!pip install click requests tqdm pyspng ninja imageio-ffmpeg==0.4.3

In [2]:
import json
import torch, pickle
import functools

from mpl_toolkits.axes_grid1 import ImageGrid
import matplotlib.pyplot as plt

import numpy as np
from tqdm import tqdm
import os

# Retrieving StyleGAN2

In order to edit the images, we need the StyleGAN2 model. This notebook assumes that stylegan2-ada-pytorch and the FFHQ pretrained weights are present in your Google Drive. If this is not the case, go to Part 1 of ImageGenerator.ipynb to download them.

cd to YOUR PATH TO stylegan2-ada-pytorch

Gurvan's path: /content/drive/MyDrive/UNIVERSITE/MVAmaterials/Projet_IIN/InterFaceGAN/stylegan2-ada-pytorch

Victor's path: /content/drive/MyDrive/Projet_IIN/InterFaceGAN/stylegan2-ada-pytorch

In [3]:
# add align_images.py
!git clone https://github.com/rkuo2000/stylegan2-ada-pytorch
%cd stylegan2-ada-pytorch

In [ ]:
if not os.path.isdir("pretrained"):
  !mkdir pretrained
  %cd pretrained
  !wget https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl
  %cd ../

In [4]:
# Retrieving Generator
device_name = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device_name)

with open('pretrained/ffhq.pkl', 'rb') as f:
    G = pickle.load(f)['G_ema'].cuda()
if device_name == "cpu":
  G.synthesis.forward = functools.partial(G.synthesis.forward, force_fp32=True)

# Retrieving labeled latent vectors

**Adjust to your setup:** Paths to labels JSON file and archive containing latent vectors

In [5]:
attribute_number =39 #[31, 39, 15, 20]
Victor = False


latent_vectors_path = '../test_ImageLatentCodes'
labels_path = f'../test_dataset/Labels/att{str(attribute_number)}_labels.json'


In [6]:
ls

 Le volume dans le lecteur C n'a pas de nom.
 Le num‚ro de s‚rie du volume est 7E6D-8E43

 R‚pertoire de C:\Users\nmadali\Nabil\GANSpace-Reimplementation-main\stylegan2-ada-pytorch-main

26/05/2024  15:10    <DIR>          .
26/05/2024  15:10    <DIR>          ..
22/09/2023  13:45    <DIR>          .github
22/09/2023  13:45                21 .gitignore
26/05/2024  15:09           796ÿ765 all_components.pdf
22/09/2023  13:45             8ÿ336 calc_metrics.py
22/09/2023  13:45            17ÿ876 dataset_tool.py
26/05/2024  14:45    <DIR>          dnnlib
22/09/2023  13:45             1ÿ196 docker_run.sh
22/09/2023  13:45               897 Dockerfile
22/09/2023  13:45    <DIR>          docs
26/05/2024  14:43       381ÿ624ÿ121 ffhq.pkl
22/09/2023  13:45             5ÿ338 generate.py
26/05/2024  15:11         1ÿ323ÿ840 layerwise.pdf
22/09/2023  13:45            16ÿ504 legacy.py
22/09/2023  13:45             4ÿ421 LICENSE.txt
25/06/2024  11:46           185ÿ530 maxime_chauve.pt
22/09/2023  13:

The following cell copies the latent vectors on the local machine as it would be much slower to manipulate vectors stored in Google Drive.

Around 4min on Colab GPU with 15k tensors.

In [7]:
# Load the labels
with open(labels_path, 'r') as f:
  labels_dict = json.load(f)

num_samples = len(labels_dict)
dim = 512
Z = torch.zeros((num_samples, dim)).to(device)
y = np.zeros(num_samples, dtype=int)

for i, (filename, label) in tqdm(enumerate(labels_dict.items())):
  sample_number = int(filename[3:-4])
  tensor_path = os.path.join(latent_vectors_path,f'tensor{sample_number}.pt')
  Z[i] = torch.load(tensor_path, map_location=device)
  y[i] = label

2000it [00:00, 3349.27it/s]


# SVM

In the $\mathcal{Z}$ latent space

In [8]:
from sklearn.svm import LinearSVC

Z_for_SVM = Z.cpu().numpy()

# Create and train the LinearSVC model
linear_svm_Z = LinearSVC()
linear_svm_Z.fit(Z_for_SVM, y)

# Get the orthogonal vector of the hyperplane made by the svm.
class_direction_Z = torch.Tensor(linear_svm_Z.coef_[0]) # a priori le vecteur est orienté de class 1 vers class 2

Z_intercept = torch.Tensor(linear_svm_Z.intercept_)

C:\Users\nmadali\AppData\Local\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


In the $\mathcal{W}$ latent space

In [9]:
W = G.mapping(Z, None)

W_for_SVM = W[:,0,:].cpu().numpy()

linear_svm_W = LinearSVC()
linear_svm_W.fit(W_for_SVM, y)

class_direction_W = torch.Tensor(linear_svm_W.coef_[0])

W_intercept = torch.Tensor(linear_svm_W.intercept_)

Setting up PyTorch plugin "bias_act_plugin"... Done.


C:\Users\nmadali\AppData\Local\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


In [21]:
attribute_number

39

Saving the normal vectors and intercepts

In [10]:
if not os.path.exists('SVM_vectors'):
    os.makedirs('SVM_vectors')

In [22]:
torch.save(class_direction_Z, f'SVM_vectors/class_direction_Z_on_3k_att{attribute_number}.pt')
torch.save(class_direction_W, f'SVM_vectors/class_direction_W_on_3k_att{attribute_number}.pt')

torch.save(Z_intercept, f'SVM_vectors/Z_intercept_on_3k_att{attribute_number}.pt')
torch.save(W_intercept, f'SVM_vectors/W_intercept_on_3k_att{attribute_number}.pt')